# Overfitting 문제 해결 

In [1]:
import tensorflow as tf
import numpy as np

## 데이터셋 준비


In [2]:
cifar10 = tf.keras.datasets.cifar10  # 32 * 32 * 3. 10 classes

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(1024)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).prefetch(1024)

## 모델 정의

In [3]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        
        self.conv1_1 = tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu')
        self.conv1_2 = tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu')
        self.pool1 = tf.keras.layers.MaxPool2D((2, 2))

        self.conv2_1 = tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu')
        self.conv2_2 = tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu')
        self.pool2 = tf.keras.layers.MaxPool2D((2, 2))

        self.conv3_1 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')
        self.conv3_2 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')
        
        self.flatten = tf.keras.layers.Flatten()        
        self.dense1 = tf.keras.layers.Dense(1024, activation='relu')
        self.dense2 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, x, training=False, mask=None):
        x = self.conv1_1(x)
        x = self.conv1_2(x)
        x = self.pool1(x)

        x = self.conv2_1(x)
        x = self.conv2_2(x)
        x = self.pool2(x)

        x = self.conv3_1(x)
        x = self.conv3_2(x)
        
        x = self.flatten(x)        
        x = self.dense1(x)
        return self.dense2(x)

## Keras API 모델 학습

In [4]:
EPOCHS = 30

In [5]:
model = MyModel()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Epoch 1/30
1563/1563 [==============================] - 11s 7ms/step - loss: 1.4232 - accuracy: 0.4837 - val_loss: 1.0775 - val_accuracy: 0.6182
Epoch 2/30
1563/1563 [==============================] - 11s 7ms/step - loss: 0.9537 - accuracy: 0.6655 - val_loss: 0.8505 - val_accuracy: 0.7072
Epoch 3/30
1563/1563 [==============================] - 11s 7ms/step - loss: 0.7433 - accuracy: 0.7394 - val_loss: 0.8370 - val_accuracy: 0.7180
Epoch 4/30
1563/1563 [==============================] - 11s 7ms/step - loss: 0.5590 - accuracy: 0.8026 - val_loss: 0.8282 - val_accuracy: 0.7262
Epoch 5/30
1563/1563 [==============================] - 11s 7ms/step - loss: 0.3811 - accuracy: 0.8658 - val_loss: 0.9516 - val_accuracy: 0.7283
Epoch 6/30
1563/1563 [==============================] - 11s 7ms/step - loss: 0.2500 - accuracy: 0.9135 - val_loss: 1.0620 - val_accuracy: 0.7244
Epoch 7/30
1563/1563 [==============================] - 11s 7ms/step - loss: 0.1762 - accuracy: 0.9391 - val_loss: 1.2898 - val_ac

# 모델 수정

In [6]:
# Layer Normalization
class ConvLNReluBlock(tf.keras.Model):
    def __init__(self, filters, kernel_size):
        super(ConvLNReluBlock, self).__init__()
        
        self.conv = tf.keras.layers.Conv2D(filters, kernel_size, padding='same', use_bias=False)
        self.LN = tf.keras.layers.LayerNormalization()
        
    def call(self, x, training=False, mask=None):
        x = self.conv(x)
        x = self.LN(x)
        out = tf.nn.relu(x)
        return out

In [9]:
class MyModel_LN(tf.keras.Model):
    def __init__(self):
        super(MyModel_LN, self).__init__()
        
        self.conv1_1 = ConvLNReluBlock(16, (3, 3))
        self.conv1_2 = ConvLNReluBlock(16, (3, 3))
        self.pool1 = tf.keras.layers.MaxPool2D((2, 2))

        self.conv2_1 = ConvLNReluBlock(32, (3, 3))
        self.conv2_2 = ConvLNReluBlock(32, (3, 3))
        self.pool2 = tf.keras.layers.MaxPool2D((2, 2))

        self.conv3_1 = ConvLNReluBlock(64, (3, 3))
        self.conv3_2 = ConvLNReluBlock(64, (3, 3))
        
        self.flatten = tf.keras.layers.Flatten()        
        
        # L2 정규화, 람다 = 0.01
        self.dense1 = tf.keras.layers.Dense(1024, activation='relu',
                                           kernel_regularizer=tf.keras.regularizers.l2(0.01))
        self.dense2 = tf.keras.layers.Dense(10, activation='softmax',
                                           kernel_regularizer=tf.keras.regularizers.l2(0.01))

    def call(self, x, training=False, mask=None):
        x = self.conv1_1(x)
        x = self.conv1_2(x)
        x = self.pool1(x)

        x = self.conv2_1(x)
        x = self.conv2_2(x)
        x = self.pool2(x)

        x = self.conv3_1(x)
        x = self.conv3_2(x)
        x = self.flatten(x)
        
        x = self.dense1(x)
        return self.dense2(x)

In [10]:
model = MyModel_LN()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Epoch 1/30
1563/1563 [==============================] - 38s 25ms/step - loss: 2.2110 - accuracy: 0.4011 - val_loss: 1.3923 - val_accuracy: 0.5405
Epoch 2/30
1563/1563 [==============================] - 38s 24ms/step - loss: 1.2702 - accuracy: 0.5893 - val_loss: 1.2571 - val_accuracy: 0.6030
Epoch 3/30
1563/1563 [==============================] - 39s 25ms/step - loss: 1.1029 - accuracy: 0.6528 - val_loss: 1.0315 - val_accuracy: 0.6814
Epoch 4/30
1563/1563 [==============================] - 38s 24ms/step - loss: 0.9986 - accuracy: 0.6913 - val_loss: 0.9748 - val_accuracy: 0.6974
Epoch 5/30
1563/1563 [==============================] - 39s 25ms/step - loss: 0.9276 - accuracy: 0.7152 - val_loss: 0.9485 - val_accuracy: 0.7141
Epoch 6/30
1563/1563 [==============================] - 38s 25ms/step - loss: 0.8804 - accuracy: 0.7342 - val_loss: 0.9025 - val_accuracy: 0.7287
Epoch 7/30
1563/1563 [==============================] - 39s 25ms/step - loss: 0.8394 - accuracy: 0.7507 - val_loss: 0.8790 -